In [ ]:
!pip install tensorflow keras numpy pandas opencv-python matplotlib seaborn


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dharshininnk","key":"1f983754839f7322c5435efce8aa3c1f"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d techsash/waste-classification-data


Dataset URL: https://www.kaggle.com/datasets/techsash/waste-classification-data
License(s): CC-BY-SA-4.0
 93% 399M/427M [00:01<00:00, 232MB/s]
100% 427M/427M [00:01<00:00, 238MB/s]


In [ ]:
!unzip waste-classification-data.zip -d data/


Archive:  waste-classification-data.zip
replace data/DATASET/TEST/O/O_12568.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


In [ ]:
train_data = train_gen.flow_from_directory(
    'data/DATASET/TRAIN',
    target_size=(224,224),
    batch_size=32,
    subset='training',
    class_mode='binary'
)

val_data = train_gen.flow_from_directory(
    'data/DATASET/TRAIN',
    target_size=(224,224),
    batch_size=32,
    subset='validation',
    class_mode='binary'
)


Found 18052 images belonging to 2 classes.
Found 4512 images belonging to 2 classes.


In [21]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)


output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)


for layer in base_model.layers:
    layer.trainable = False


model.compile(optimizer=Adam(1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])


history = model.fit(train_data, validation_data=val_data, epochs=5)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
565/565 ━━━━━━━━━━━━━━━━━━━━ 1072s 2s/step - accuracy: 0.8268 - loss: 0.3801 - val_accuracy: 0.9240 - val_loss: 0.2014
Epoch 2/5
565/565 ━━━━━━━━━━━━━━━━━━━━ 1113s 2s/step - accuracy: 0.9218 - loss: 0.2073 - val_accuracy: 0.9357 - val_loss: 0.1730
Epoch 3/5
565/565 ━━━━━━━━━━━━━━━━━━━━ 1068s 2s/step - accuracy: 0.9206 - loss: 0.1990 - val_accuracy: 0.9300 - val_loss: 0.1929
Epoch 4/5
565/565 ━━━━━━━━━━━━━━━━━━━━ 1042s 2s/step - accuracy: 0.9270 - loss: 0.1851 - val_accuracy: 0.9351 - val_loss: 0.1814
Epoch 5/5
565/565 ━━━━━━━━━━━━━━━━━━━━ 1109s 2s/step - accuracy: 0.9350 - loss: 0.1679 - val_accuracy: 0.9251 - val_loss: 0.1996


In [23]:
model.save('waste_model.h5')

In [24]:
from tensorflow.keras.models import load_model
model = load_model('waste_model.h5')


In [ ]:
pred = model.predict(x)


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [25]:
%%writefile app.py
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image

model = load_model('waste_model.h5')

st.title("♻️ Waste Classification for Smart Recycling")
st.write("Upload an image of waste to classify it as Organic or Recyclable.")

uploaded_file = st.file_uploader("Choose an image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    img = Image.open(uploaded_file).convert("RGB")
    st.image(img, caption="Uploaded Image", use_container_width=True)

    img = img.resize((224, 224))
    x = np.array(img) / 255.0
    x = np.expand_dims(x, axis=0)

    pred = model.predict(x)[0][0]

    if pred > 0.5:
        st.success("♻️ Predicted: Recyclable Waste")
    else:
        st.warning("🍎 Predicted: Organic Waste")


Writing app.py


In [26]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.7 MB/s eta 0:00:00


In [27]:
from pyngrok import ngrok

!ngrok config add-authtoken 350WsQUMWftJEXmLMsdCloxG60e_4FG7X33vDhRCox5QmXuFZ


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [28]:
from pyngrok import ngrok


public_url = ngrok.connect(8501)
print("🔗 Your Streamlit app link:", public_url)

!streamlit run app.py &>/dev/null&


🔗 Your Streamlit app link: NgrokTunnel: "https://avoidable-suturally-han.ngrok-free.dev" -> "http://localhost:8501"
